In [1]:
import sys
sys.path.insert(0, '../scripts/')
from helper_functions import convert_census_to_postcode

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

import statsmodels.api as sm
from statsmodels.formula.api import ols

import geopandas as gpd

from scipy.interpolate import interp1d

output_dir = '../data/curated/'


<h3> Preparation for forecasting </h3>

In [2]:
properties_df = pd.read_csv("../data/curated/properties_processed.csv", index_col=0)
properties_df['Postcode'] = properties_df['Postcode'].astype(str)

# read in census dataframes
census_df = pd.read_csv("../data/curated/census_data.csv")
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")

In [4]:
census_df_postcode_agg = convert_census_to_postcode(census_df, sa2_postcode_map, 'mean_no_zero')

Median Rental Prices

In [7]:
median_rent = properties_df.groupby(["Postcode"])["Cost"].median()


In [8]:
median_rent.to_csv(f'{output_dir}median_rental_postcode.csv')

In [5]:
i = 0 
impute_features = ['tot_population', 'avg_med_mortg_rep', 'avg_med_person_inc', 'avg_med_rent', 'avg_med_hh_inc', 'tot_avg_hh_size']
init_years = [11, 16, 21]
new_years = list(range(11, 22))


for i in range(0, len(census_df_postcode_agg)):
    data = census_df_postcode_agg.iloc[i]
    #print(census_df_postcode_agg.iloc[i])

    # interpolation 
    interpolated_dict = dict()
    interpolated_dict['date']= [f'20{year}' for year in new_years]

    for feature in impute_features:
        init_y = data[[f'{feature}_{init_years[0]}', f'{feature}_{init_years[1]}', f'{feature}_{init_years[2]}']].tolist()
        #print(init_y)

        interp_func = interp1d(init_years, init_y)
        new_y = list(interp_func(new_years))
        #print(new_y)

        interpolated_dict[feature] = new_y

    # creation of new dataframe
    interpolated_df = pd.DataFrame(interpolated_dict)
    
    # model
    interpolated_df['date'] = pd.to_datetime(interpolated_df['date'])
    interpolated_df = interpolated_df.set_index('date')
    interpolated_df = interpolated_df.asfreq('YS') # frequency = year started
    interpolated_df = interpolated_df.sort_index()
    
    steps = 2
    data_train = interpolated_df[:int(0.8 * (len(interpolated_df)))]
    data_test = interpolated_df[int(0.8 * (len(interpolated_df))):]

    # fit model 
    model = VAR(endog = data_train)
    model_fit = model.fit()

    prediction = model_fit.forecast(model_fit.y, steps = len(data_test)) 



    break 


,tot_population,avg_med_mortg_rep,avg_med_person_inc,avg_med_rent,avg_med_hh_inc,tot_avg_hh_size
date,,,,,,
2011-01-01,124551.0,2213.380,862.180,447.060,1896.760,1.970
2012-01-01,133074.0,2178.780,1786.508,436.800,1813.914,1.952
2013-01-01,141597.0,2144.180,2710.836,426.540,1731.068,1.934
2014-01-01,150120.0,2109.580,3635.164,416.280,1648.222,1.916
2015-01-01,158643.0,2074.980,4559.492,406.020,1565.376,1.898
2016-01-01,167166.0,2040.380,5483.820,395.760,1482.530,1.880
2017-01-01,169417.6,2040.342,5680.608,400.246,1617.906,1.876
2018-01-01,171669.2,2040.304,5877.396,404.732,1753.282,1.872
2019-01-01,173920.8,2040.266,6074.184,409.218,1888.658,1.868
